
# California Housing — Predicting Community Median Income (CRISP‑DM, scikit‑learn)

This notebook is a **textbook‑quality CRISP‑DM walkthrough** using the Kaggle dataset  
**“California Housing Prices”** (by *camnugent*). We **predict `median_income`** from the
remaining variables (location, housing stock, demographics). The pipeline is **compute‑aware**
and designed for **Google Colab**.

**Phases covered**
- Business Understanding → Data Understanding → Data Preparation (cleaning, preprocessing)
- Feature Engineering & Selection → Outlier analysis/processing → Clustering (unsupervised segmentation)
- Baselines & Model Benchmarking (geo‑aware CV) → Final Holdout Evaluation & Explainability
- Final Recommendation, Deployment/Monitoring notes

> Units: `median_income` is approximately **$10,000s of 1990 USD**. We report MAE in model units and dollars.


finished


## 1) Environment Setup (Colab)
- Installs: `scikit-learn`, `shap` (optional), and `kaggle` (for dataset download).
- Sets a fast/compute‑aware configuration.


In [ ]:

# If running on Colab, you can keep these. On local, adjust as needed.
!pip -q install -U scikit-learn shap kaggle

FAST = True                 # Set to False for more exhaustive tuning
RANDOM_SEED = 42
N_JOBS = -1
CV_SPLITS = 5

# Compact, compute-aware defaults
LINEAR_N_ITER = 15 if FAST else 25
RF_N_ITER     = 16 if FAST else 32
HGB_N_ITER    = 20 if FAST else 40
BOOTSTRAP_B   = 1000 if FAST else 2000


finished


## 2) Data Access (Kaggle or Manual Upload)
Preferred: **Kaggle API**. Alternative: manual upload of `housing.csv`.


In [ ]:

import os, zipfile, pathlib, json

DATA_DIR = "/content/data"
os.makedirs(DATA_DIR, exist_ok=True)
CSV_PATH = os.path.join(DATA_DIR, "housing.csv")

if not os.path.exists(CSV_PATH):
    kaggle_creds = os.path.expanduser("~/.kaggle/kaggle.json")
    if os.path.exists(kaggle_creds):
        print("Kaggle credentials found. Downloading dataset...")
        # Requires Kaggle API to be configured (kaggle.json placed at ~/.kaggle with correct permissions).
        # Dataset: camnugent/california-housing-prices
        !kaggle datasets download -d camnugent/california-housing-prices -p $DATA_DIR --unzip
    else:
        print("Kaggle credentials not found. Please either:")
        print("  1) Upload ~/.kaggle/kaggle.json and re-run this cell, or")
        print("  2) Use the next cell to upload housing.csv manually.")


In [ ]:

# Manual upload (Colab). Run this cell if the CSV is not already present.
# After upload, if the file isn't named 'housing.csv', we place/rename it accordingly.
import os
if not os.path.exists(CSV_PATH):
    try:
        from google.colab import files
        uploaded = files.upload()
        for fn in uploaded.keys():
            if fn.lower().endswith(".csv"):
                os.replace(fn, CSV_PATH)
                print("Saved uploaded CSV to", CSV_PATH)
    except Exception as e:
        print("Manual upload not available in this environment:", e)


finished


## 3) Imports & Global Helpers


In [ ]:

import warnings, math
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from pathlib import Path
from dataclasses import dataclass

from sklearn import set_config
set_config(transform_output="default")

RNG = np.random.RandomState(RANDOM_SEED)

def to_usd(x): return x * 10_000  # convert model units (~$10k) to dollars


finished


## 4) Data Understanding — Load & Initial Audit
- Confirm shape, dtypes, missingness, duplicates.
- Sanity checks on coordinates and logical constraints.


In [ ]:

df_raw = pd.read_csv(CSV_PATH)
df = df_raw.copy()

print("Shape:", df.shape)
print(df.dtypes)
print(f"Memory MB: {df.memory_usage(deep=True).sum()/1_048_576:.2f}")
display(df.head(3))
display(df.describe())
display(df.describe(include='object'))
print("Missing counts:\n", df.isna().sum().sort_values(ascending=False))
print("Duplicates:", df.duplicated().sum())

# Quick target profile
t = df['median_income'].agg(['min','max','mean','median','std'])
print("Target stats (median_income; ~$10k units):\n", t)
print("Skew:", df['median_income'].skew())


finished


## 5) Cleaning (minimal, auditable)
- Drop out-of-CA coordinates and impossible/illogical rows.
- Keep missing `total_bedrooms` to impute later.
- Cast `ocean_proximity` to category; drop duplicates.


In [ ]:

def clean_raw(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Bounds for California
    lat_ok = df['latitude'].between(32, 42.5)
    lon_ok = df['longitude'].between(-125, -113)
    df = df[lat_ok & lon_ok]
    # Logical constraints
    df = df[df['households'] > 0]
    df = df[df['total_rooms'] > 0]
    df = df[df['total_bedrooms'].isna() | (df['total_bedrooms'] >= 0)]
    df = df[df['population'] >= df['households']]
    df = df[(df['total_bedrooms'].isna()) | (df['total_rooms'] >= df['total_bedrooms'])]
    # Types & duplicates
    if df['ocean_proximity'].dtype.name != 'category':
        df['ocean_proximity'] = df['ocean_proximity'].astype('category')
    df = df.drop_duplicates()
    return df

df = clean_raw(df)
print("After cleaning:", df.shape)
print(df.isna().sum().sort_values(ascending=False).head(3))


finished


## 6) Feature Engineering & Robust Transforms
- Engineered ratios: rooms/household, bedrooms/room, population/household.
- Winsorization for heavy tails; log1p for large counts.
- Light spatial polynomials (`lat^2`, `lon^2`, `lat*lon`).


In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin

TARGET = 'median_income'
CORE_BASE = ['longitude','latitude','housing_median_age',
             'total_rooms','total_bedrooms','population','households','ocean_proximity']

class AddRatios(BaseEstimator, TransformerMixin):
    def __init__(self, eps=1e-6): self.eps = eps
    def fit(self, X, y=None): return self
    def transform(self, X):
        X = X.copy()
        X['rooms_per_household']      = X['total_rooms']    / (X['households'] + self.eps)
        X['bedrooms_per_room']        = X['total_bedrooms'] / (X['total_rooms'] + self.eps)
        X['population_per_household'] = X['population']     / (X['households'] + self.eps)
        X['lat2']   = X['latitude']**2
        X['lon2']   = X['longitude']**2
        X['lat_lon']= X['latitude']*X['longitude']
        return X

RATIO_COLS = ['rooms_per_household','bedrooms_per_room','population_per_household','lat2','lon2','lat_lon']
NUM_BASE   = ['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households']
CAT_COLS   = ['ocean_proximity']
NUM_ALL    = NUM_BASE + RATIO_COLS

class Winsorize(BaseEstimator, TransformerMixin):
    def __init__(self, cols, lower=0.005, upper=0.995):
        self.cols, self.lower, self.upper = cols, lower, upper
    def fit(self, X, y=None):
        X = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X, columns=self.cols)
        self.bounds_ = {c: (X[c].quantile(self.lower), X[c].quantile(self.upper)) for c in self.cols if c in X}
        return self
    def transform(self, X):
        X = X.copy()
        for c,(lo,hi) in self.bounds_.items():
            if c in X: X[c] = X[c].clip(lo, hi)
        if 'bedrooms_per_room' in X:
            X['bedrooms_per_room'] = X['bedrooms_per_room'].clip(0, 1)
        return X

class Log1pCols(BaseEstimator, TransformerMixin):
    def __init__(self, cols): self.cols = cols
    def fit(self, X, y=None): return self
    def transform(self, X):
        X = X.copy()
        for c in self.cols:
            if c in X: X[c] = np.log1p(np.maximum(X[c], 0))
        return X


finished


## 7) Preprocessing Pipelines (Linear vs Tree)
- Linear: median impute + standardize; Tree: median impute only.
- Categorical: `OneHotEncoder(handle_unknown='ignore')` (dense).


In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def make_ohe():
    # Backward-compatible OHE for older scikit-learn
    try:
        return OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown='ignore', sparse=False)

cat_common = Pipeline([('impute', SimpleImputer(strategy='most_frequent')),
                       ('onehot', make_ohe())])

num_linear = Pipeline([('impute', SimpleImputer(strategy='median')),
                       ('scale',  StandardScaler())])

num_tree   = SimpleImputer(strategy='median')

preproc_linear = ColumnTransformer([
    ('num', num_linear, NUM_ALL),
    ('cat', cat_common, CAT_COLS)
], remainder='drop')

preproc_tree = ColumnTransformer([
    ('num', num_tree, NUM_ALL),
    ('cat', cat_common, CAT_COLS)
], remainder='drop')

# Full feature pipelines
clip_cols = NUM_BASE + RATIO_COLS
log_cols  = ['total_rooms','total_bedrooms','population','households']

pipe_linear_features = Pipeline([
    ('add',    AddRatios()),
    ('winsor', Winsorize(cols=clip_cols, lower=0.005, upper=0.995)),
    ('log1p',  Log1pCols(cols=log_cols)),
    ('prep',   preproc_linear)
])

pipe_tree_features = Pipeline([
    ('add',    AddRatios()),
    ('winsor', Winsorize(cols=clip_cols, lower=0.005, upper=0.995)),
    ('prep',   preproc_tree)
])


finished


## 8) Geographic Split (Leakage‑aware)
We create 0.5° lat‑lon tiles and use them as **groups** for GroupKFold CV and
to form a **geographically disjoint** test set.


In [ ]:

from sklearn.model_selection import GroupKFold

def make_geo_groups(df, tile_deg=0.5):
    lat_bin = np.floor(df['latitude']  / tile_deg).astype(int)
    lon_bin = np.floor(df['longitude'] / tile_deg).astype(int)
    return (lat_bin.astype(str) + "_" + lon_bin.astype(str)).astype('category')

groups = make_geo_groups(df, tile_deg=0.5)
unique_groups = pd.Series(groups).unique()

RNG = np.random.RandomState(RANDOM_SEED)
test_groups = RNG.choice(unique_groups, size=int(np.ceil(0.20*len(unique_groups))), replace=False)
is_test = groups.isin(test_groups)

X_core = df[['longitude','latitude','housing_median_age','total_rooms','total_bedrooms',
             'population','households','ocean_proximity']].copy()
y = df[TARGET].copy()

X_train, X_test = X_core[~is_test], X_core[is_test]
y_train, y_test = y[~is_test], y[is_test]
g_train, g_test = groups[~is_test], groups[is_test]

gkf = GroupKFold(n_splits=CV_SPLITS)
print("Train/Test sizes:", X_train.shape, X_test.shape)
print("Unique test tiles:", len(pd.unique(g_test)))
assert set(pd.unique(g_train)).isdisjoint(set(pd.unique(g_test)))


finished


## 9) Outlier Analysis & Mutual Information (quick)
We flag univariate outliers (IQR/MAD) and compute **mutual information** with the target
on preprocessed features to inform selection.


In [ ]:

from scipy.stats import median_abs_deviation
from sklearn.feature_selection import mutual_info_regression

def mad_zscore(s):
    med = np.nanmedian(s); mad = median_abs_deviation(s, nan_policy='omit', scale='normal')
    return (s - med) / (mad if mad>0 else 1.0)

NUM_SIMPLE = ['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households']
flags = {}
for c in NUM_SIMPLE:
    s = df[c]
    iqr = s.quantile(0.75) - s.quantile(0.25)
    iqr_mask = (s < s.quantile(0.25) - 1.5*iqr) | (s > s.quantile(0.75) + 1.5*iqr)
    mz_mask  = mad_zscore(s).abs() > 3.5
    flags[c] = dict(iqr=float(iqr_mask.mean()), mad=float(mz_mask.mean()))
pd.DataFrame(flags).T.sort_values('mad', ascending=False)

# Mutual information on train
Xt = pipe_linear_features.fit_transform(X_train, y_train)
feat_names = pipe_linear_features.named_steps['prep'].get_feature_names_out()
mi = mutual_info_regression(Xt, y_train, random_state=RANDOM_SEED)
mi_tbl = pd.DataFrame({'feature': feat_names, 'MI': mi}).sort_values('MI', ascending=False)
mi_tbl.head(15)


finished


## 10) Clustering (structure discovery)
We use **MiniBatchKMeans** on scaled engineered features to derive an unsupervised
`cluster_id` for segmentation and potential modeling gains.


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

CLUST_COLS = ['latitude','longitude','housing_median_age',
              'rooms_per_household','bedrooms_per_room','population_per_household']

# Prepare train-only engineered features for clustering
prep_for_cluster = Pipeline([
    ('add',    AddRatios()),
    ('winsor', Winsorize(cols=NUM_BASE + RATIO_COLS, lower=0.005, upper=0.995))
])

Xtr_eng = prep_for_cluster.fit_transform(X_train.copy())
Xs = StandardScaler().fit_transform(Xtr_eng[CLUST_COLS])

# Pick k via silhouette on a sample
idx = RNG.choice(len(Xs), size=min(5000, len(Xs)), replace=False)
results = []
for k in range(3, 11):
    km = MiniBatchKMeans(n_clusters=k, batch_size=1024, random_state=RANDOM_SEED, n_init=10)
    lbl = km.fit_predict(Xs)
    sil = silhouette_score(Xs[idx], lbl[idx])
    inertia = km.inertia_
    results.append((k, sil, inertia))

sil_tbl = pd.DataFrame(results, columns=['k','silhouette','inertia']).sort_values('silhouette', ascending=False)
display(sil_tbl)

BEST_K = int(sil_tbl.iloc[0]['k']) if len(sil_tbl) else 6
print("Chosen k:", BEST_K)

# Fit final kmeans on train
scaler = StandardScaler().fit(Xtr_eng[CLUST_COLS])
kmeans = MiniBatchKMeans(n_clusters=BEST_K, batch_size=1024, random_state=RANDOM_SEED, n_init=10).fit(scaler.transform(Xtr_eng[CLUST_COLS]))

# Assign labels
X_train_cl = X_train.copy()
X_train_cl['cluster_id'] = pd.Categorical(kmeans.predict(scaler.transform(Xtr_eng[CLUST_COLS])))

Xte_eng = prep_for_cluster.transform(X_test.copy())
X_test_cl = X_test.copy()
X_test_cl['cluster_id'] = pd.Categorical(kmeans.predict(scaler.transform(Xte_eng[CLUST_COLS])))

# Inspect clusters (centroids approx inverse-scaled)
centroids = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=CLUST_COLS)
centroids['cluster_id'] = range(BEST_K)
display(centroids.head())


finished


## 11) Cluster‑aware Preprocessing (optional)


In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin

class KMeansClusterer(BaseEstimator, TransformerMixin):
    def __init__(self, cols, n_clusters=6, random_state=42):
        self.cols, self.n_clusters, self.random_state = cols, n_clusters, random_state
    def fit(self, X, y=None):
        Xp = prep_for_cluster.fit_transform(X)
        self.scaler_ = StandardScaler().fit(Xp[self.cols])
        self.km_ = MiniBatchKMeans(n_clusters=self.n_clusters, batch_size=1024,
                                   random_state=self.random_state, n_init=10).fit(self.scaler_.transform(Xp[self.cols]))
        return self
    def transform(self, X):
        X = X.copy()
        Xp = prep_for_cluster.transform(X)
        lbl = self.km_.predict(self.scaler_.transform(Xp[self.cols]))
        X['cluster_id'] = pd.Categorical(lbl)
        return X

CAT_WITH_CLUSTER = ['ocean_proximity','cluster_id']

cat_common2 = Pipeline([('impute', SimpleImputer(strategy='most_frequent')),
                        ('onehot', make_ohe())])

preproc_linear_with_cluster = ColumnTransformer([
    ('num', Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())]), NUM_ALL),
    ('cat', cat_common2, CAT_WITH_CLUSTER)
])


finished


## 12) Baselines & Model Benchmarking (geo‑aware CV)
Primary: **MAE**. Secondary: **RMSE**, **R²**. Uses **GroupKFold** over geographic tiles.


In [ ]:

from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from scipy.stats import loguniform, uniform

cv_splits = list(gkf.split(X_train, y_train, groups=g_train))
SCORING = {'mae': 'neg_mean_absolute_error',
           'mse': 'neg_mean_squared_error',
           'r2':  'r2'}

def summarize_cv(cv_out, name):
    mae  = -cv_out['test_mae']
    rmse = np.sqrt(-cv_out['test_mse'])
    return dict(model=name,
                MAE_mean=mae.mean(), MAE_std=mae.std(),
                RMSE_mean=rmse.mean(), RMSE_std=rmse.std(),
                R2_mean=cv_out['test_r2'].mean(), R2_std=cv_out['test_r2'].std())

results = []

# Dummy baseline
m_dummy = DummyRegressor(strategy='mean')
cv = cross_validate(m_dummy, X_train, y_train, cv=cv_splits, scoring=SCORING)
results.append(summarize_cv(cv, "Dummy"))

# Linear baseline
m_lr = Pipeline([('X', pipe_linear_features), ('lr', LinearRegression())])
cv = cross_validate(m_lr, X_train, y_train, cv=cv_splits, scoring=SCORING)
results.append(summarize_cv(cv, "Linear (no cluster)"))

# Cluster-augmented linear baseline
m_lr_cl = Pipeline([
    ('cluster', KMeansClusterer(cols=CLUST_COLS, n_clusters=BEST_K, random_state=RANDOM_SEED)),
    ('add',     AddRatios()),
    ('winsor',  Winsorize(cols=NUM_BASE + RATIO_COLS, lower=0.005, upper=0.995)),
    ('log1p',   Log1pCols(cols=['total_rooms','total_bedrooms','population','households'])),
    ('prep',    preproc_linear_with_cluster),
    ('lr',      LinearRegression())
])
cv = cross_validate(m_lr_cl, X_train, y_train, cv=cv_splits, scoring=SCORING)
results.append(summarize_cv(cv, "Linear +Cluster"))

# Helper to tune linear family
def tune_linear(base_estimator, param_grid, name, n_iter=LINEAR_N_ITER):
    pipe = Pipeline([('X', pipe_linear_features), (name, base_estimator)])
    search = RandomizedSearchCV(
        pipe, param_distributions=param_grid, n_iter=n_iter,
        cv=cv_splits, scoring='neg_mean_absolute_error',
        n_jobs=N_JOBS, random_state=RANDOM_SEED, refit=True
    )
    search.fit(X_train, y_train)
    cv = cross_validate(search.best_estimator_, X_train, y_train, cv=cv_splits, scoring=SCORING)
    res = summarize_cv(cv, f"{name} (best)")
    res['best_params'] = search.best_params_
    return search, res

# Regularized linear models
ridge_search, res_ridge = tune_linear(Ridge(), { 'Ridge__alpha': loguniform(1e-3, 1e3) }, 'Ridge')
lasso_search, res_lasso = tune_linear(Lasso(max_iter=10000), { 'Lasso__alpha': loguniform(1e-4, 1e1) }, 'Lasso')
enet_search,  res_enet  = tune_linear(ElasticNet(max_iter=10000),
                                      { 'ElasticNet__alpha': loguniform(1e-4, 1e1),
                                        'ElasticNet__l1_ratio': uniform(0.05, 0.9) }, 'ElasticNet')

results += [res_ridge, res_lasso, res_enet]

# Random Forest
rf_pipe = Pipeline([('X', pipe_tree_features),
                    ('rf', RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=N_JOBS))])

rf_grid = {
    'rf__n_estimators': [200, 300, 400, 600],
    'rf__max_depth': [None, 10, 15, 20],
    'rf__min_samples_leaf': [1, 2, 4, 8],
    'rf__max_features': [0.5, 0.7, 0.9, 1.0]
}
rf_search = RandomizedSearchCV(rf_pipe, rf_grid, n_iter=RF_N_ITER, cv=cv_splits,
                               scoring='neg_mean_absolute_error',
                               n_jobs=N_JOBS, random_state=RANDOM_SEED, refit=True)
rf_search.fit(X_train, y_train)
cv = cross_validate(rf_search.best_estimator_, X_train, y_train, cv=cv_splits, scoring=SCORING)
res_rf = summarize_cv(cv, "RandomForest (best)")
res_rf['best_params'] = rf_search.best_params_
results.append(res_rf)

# HistGradientBoosting
hgb_pipe = Pipeline([('X', pipe_tree_features),
                     ('hgb', HistGradientBoostingRegressor(random_state=RANDOM_SEED,
                                                           early_stopping=True, validation_fraction=0.1))])

from scipy.stats import loguniform as logu
hgb_grid = {
    'hgb__learning_rate': logu(0.03, 0.5),
    'hgb__max_depth': [None, 6, 8, 12],
    'hgb__min_samples_leaf': [10, 20, 30, 50, 60],
    'hgb__l2_regularization': logu(1e-10, 1e-2),
    'hgb__max_bins': [128, 255]
}
hgb_search = RandomizedSearchCV(hgb_pipe, hgb_grid, n_iter=HGB_N_ITER, cv=cv_splits,
                                scoring='neg_mean_absolute_error',
                                n_jobs=N_JOBS, random_state=RANDOM_SEED, refit=True)
hgb_search.fit(X_train, y_train)
cv = cross_validate(hgb_search.best_estimator_, X_train, y_train, cv=cv_splits, scoring=SCORING)
res_hgb = summarize_cv(cv, "HistGB (best)")
res_hgb['best_params'] = hgb_search.best_params_
results.append(res_hgb)

res_df = pd.DataFrame(results).sort_values('MAE_mean')
display(res_df[['model','MAE_mean','MAE_std','RMSE_mean','R2_mean']])

candidates = {
    'Ridge': ridge_search.best_estimator_,
    'Lasso': lasso_search.best_estimator_,
    'ElasticNet': enet_search.best_estimator_,
    'RandomForest': rf_search.best_estimator_,
    'HistGB': hgb_search.best_estimator_,
    'Linear (no cluster)': m_lr,
    'Linear +Cluster': m_lr_cl
}
champion_name = res_df.iloc[0]['model']
# Normalize key name before " (best)"
champ_key = champion_name.split(' (')[0]
champion = candidates.get(champ_key, hgb_search.best_estimator_)
print("Champion by CV MAE:", champion_name)


finished


## 13) Final Training on Train Tiles & Holdout Evaluation on Disjoint Test Tiles
We compare the champion to **Dummy** and a **Linear baseline**.
We also produce 95% **bootstrap CIs** for MAE/RMSE/R².


In [ ]:

from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor

# Refit
champion.fit(X_train, y_train)
dummy = DummyRegressor(strategy='mean').fit(X_train, y_train)
linear_baseline = Pipeline([('X', pipe_linear_features), ('lr', LinearRegression())]).fit(X_train, y_train)

# Predictions
y_pred = champion.predict(X_test)
y_dummy = dummy.predict(X_test)
y_lin   = linear_baseline.predict(X_test)

def reg_metrics(y_true, y_hat):
    mae  = mean_absolute_error(y_true, y_hat)
    rmse = sqrt(mean_squared_error(y_true, y_hat))
    r2   = r2_score(y_true, y_hat)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

test_metrics = {'Champion': reg_metrics(y_test, y_pred),
                'Dummy':    reg_metrics(y_test, y_dummy),
                'Linear':   reg_metrics(y_test, y_lin)}
pd.DataFrame(test_metrics).T


In [ ]:

# Bootstrap CIs (compute-aware)
RNG = np.random.RandomState(RANDOM_SEED)

def bootstrap_ci(y_true, y_hat, fn, B=BOOTSTRAP_B, alpha=0.05):
    n = len(y_true); stats = np.empty(B)
    for b in range(B):
        idx = RNG.randint(0, n, n)
        stats[b] = fn(y_true[idx], y_hat[idx])
    lo, hi = np.quantile(stats, [alpha/2, 1-alpha/2])
    return stats.mean(), (lo, hi)

mae_mean, (mae_lo, mae_hi) = bootstrap_ci(y_test.values, y_pred, lambda a,b: mean_absolute_error(a,b))
rmse_mean, (rmse_lo, rmse_hi) = bootstrap_ci(y_test.values, y_pred, lambda a,b: np.sqrt(mean_squared_error(a,b)))
r2_mean, (r2_lo, r2_hi) = bootstrap_ci(y_test.values, y_pred, lambda a,b: r2_score(a,b))

print(f"Champion MAE:  {mae_mean:.3f} [{mae_lo:.3f}, {mae_hi:.3f}]  (~${to_usd(mae_mean):,.0f})")
print(f"Champion RMSE: {rmse_mean:.3f} [{rmse_lo:.3f}, {rmse_hi:.3f}]")
print(f"Champion R²:   {r2_mean:.3f} [{r2_lo:.3f}, {r2_hi:.3f}]")


finished


## 14) Diagnostics & Calibration
Residual histograms, residuals vs. fitted and latitude, and a reliability curve.


In [ ]:

res = y_test - y_pred
yp  = y_pred

# Residual histogram
plt.figure(); plt.hist(res, bins=50)
plt.title("Residuals (y_true - y_pred)"); plt.xlabel("residual"); plt.ylabel("count"); plt.tight_layout(); plt.show()

# Residuals vs fitted
plt.figure(); plt.scatter(yp, res, s=6, alpha=0.5)
plt.axhline(0, ls='--'); plt.xlabel("predicted"); plt.ylabel("residual"); plt.title("Residuals vs Predicted")
plt.tight_layout(); plt.show()

# Residuals vs latitude (spatial drift)
plt.figure(); plt.scatter(X_test['latitude'], res, s=6, alpha=0.5)
plt.axhline(0, ls='--'); plt.xlabel("latitude"); plt.ylabel("residual"); plt.title("Residuals vs Latitude")
plt.tight_layout(); plt.show()

# Reliability curve
bins = np.quantile(yp, np.linspace(0,1,11))
cats = pd.cut(yp, bins=bins, include_lowest=True)
rel = pd.DataFrame({'y_true': y_test, 'y_pred': yp, 'bin': cats}).groupby('bin').mean()
plt.figure(); plt.plot(rel['y_pred'].values, rel['y_true'].values, marker='o')
mn, mx = rel.to_numpy().min(), rel.to_numpy().max()
plt.plot([mn, mx], [mn, mx], ls='--')
plt.xlabel("mean predicted"); plt.ylabel("mean actual"); plt.title("Reliability (Regression)")
plt.tight_layout(); plt.show()


finished


## 15) Slice Metrics (error parity)
Error by `ocean_proximity` and by income deciles.


In [ ]:

def mae_rmse(y_true, y_hat):
    return pd.Series({'MAE': mean_absolute_error(y_true, y_hat),
                      'RMSE': sqrt(mean_squared_error(y_true, y_hat))})

# By ocean proximity
slice_tbl = (pd.DataFrame({'y': y_test, 'yhat': y_pred, 'grp': X_test['ocean_proximity'].astype('category')})
             .groupby('grp').apply(lambda g: mae_rmse(g['y'], g['yhat'])))
display(slice_tbl)

# By income decile
dec = pd.qcut(y_test, 10, labels=False, duplicates='drop')
dec_tbl = (pd.DataFrame({'y': y_test, 'yhat': y_pred, 'dec': dec})
           .groupby('dec').apply(lambda g: mae_rmse(g['y'], g['yhat'])))
display(dec_tbl)


finished


## 16) Explainability: Permutation Importance (test set) and optional SHAP


In [ ]:

from sklearn.inspection import permutation_importance

def find_column_transformer(pl):
    # Try common step names
    for name in ['X', 'feat']:
        if name in getattr(pl, 'named_steps', {}):
            step = pl.named_steps[name]
            if hasattr(step, 'named_steps') and 'prep' in step.named_steps:
                return step.named_steps['prep']
    # Fallback: search any ColumnTransformer in pipeline
    for step_name, step_obj in getattr(pl, 'named_steps', {}).items():
        if "ColumnTransformer" in type(step_obj).__name__:
            return step_obj
    return None

prep = find_column_transformer(champion)
feat_names = None
if prep is not None and hasattr(prep, 'get_feature_names_out'):
    try:
        feat_names = prep.get_feature_names_out()
    except Exception:
        pass

perm = permutation_importance(champion, X_test, y_test, n_repeats=5, random_state=RANDOM_SEED,
                              scoring='neg_mean_absolute_error')
imp = pd.DataFrame({'feature': np.arange(len(perm.importances_mean)) if feat_names is None else feat_names,
                    'perm_importance': perm.importances_mean}).sort_values('perm_importance', ascending=False)
display(imp.head(20))

# Bar plot
top = imp.head(15)
plt.figure(); plt.barh(top['feature'][::-1], top['perm_importance'][::-1])
plt.title('Permutation importance (MAE decrease, test set)'); plt.tight_layout(); plt.show()

# Optional SHAP for tree-based champions; may not run depending on pipeline structure.
try:
    import shap
    # Attempt to find an underlying estimator that SHAP can handle; otherwise try the whole pipeline.
    estimator = champion
    sample_n = min(1000, X_test.shape[0])
    samp_idx = np.random.RandomState(RANDOM_SEED).choice(X_test.index, size=sample_n, replace=False)
    Xs = X_test.loc[samp_idx]
    explainer = shap.Explainer(estimator)
    sv = explainer(Xs)
    shap.plots.beeswarm(sv, max_display=15)
except Exception as e:
    print("SHAP not run (optional):", e)


finished


## 17) Summary: Gains vs Baselines


In [ ]:

def pct_improve(a, b):  # improvement from baseline a to model b
    return 100.0 * (a - b) / a

MAE_c = mean_absolute_error(y_test, y_pred)
MAE_d = mean_absolute_error(y_test, y_dummy)
MAE_l = mean_absolute_error(y_test, y_lin)

summary = pd.DataFrame({
    'Model': ['Champion','Linear baseline','Dummy baseline'],
    'MAE (~$10k)': [MAE_c, MAE_l, MAE_d],
    'MAE ($)': [to_usd(MAE_c), to_usd(MAE_l), to_usd(MAE_d)]
}).set_index('Model')

summary.loc['Gain vs Dummy','% MAE improv']  = pct_improve(MAE_d, MAE_c)
summary.loc['Gain vs Linear','% MAE improv'] = pct_improve(MAE_l, MAE_c)
summary


finished


## 18) Save Artifacts (model, report)


In [ ]:

import os, joblib, json, time
ART_DIR = "/content/models"
os.makedirs(ART_DIR, exist_ok=True)

joblib.dump(champion, os.path.join(ART_DIR, "champion_model.joblib"))
with open(os.path.join(ART_DIR, "metrics_test.json"), "w") as f:
    json.dump({k: {m: float(v[m]) for m in v} for k,v in test_metrics.items()}, f, indent=2)

print("Saved to:", ART_DIR)


finished


## 19) Inference Example (data contract)
Inputs required (columns):  
`longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, ocean_proximity`  
Output: predicted `median_income` (~$10k) and dollars.


In [ ]:

sample = X_test.iloc[[0]].copy()
pred_unit = float(champion.predict(sample)[0])
print("Predicted median_income (~$10k):", round(pred_unit, 3))
print("Predicted median_income ($):", f"${to_usd(pred_unit):,.0f}")


finished


## Appendix — Data Dictionary (selected)
- **longitude, latitude**: location (float)
- **housing_median_age**: median age of houses (years)
- **total_rooms, total_bedrooms**: stock counts
- **population, households**: demographic counts
- **ocean_proximity**: categorical (`<1H OCEAN`, `INLAND`, `ISLAND`, `NEAR BAY`, `NEAR OCEAN`)
- **median_income**: target (~$10k of 1990 USD)  


finished